In [3]:
from langchain_google_genai import GoogleGenerativeAI, ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory, ChatMessageHistory
from langchain.prompts import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.document_loaders.csv_loader import CSVLoader
from dotenv import load_dotenv
import os
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from langchain.vectorstores import Qdrant

# Chargement des variables d'environnement
load_dotenv()

# Utilisation du modèle d'embeddings GoogleGenerativeAI
def chunk_embedder():
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", temperature=0.5)
    return embeddings

def load_llm():
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)
    return llm

def init_chat():
    llm = load_llm()
    embeddings = chunk_embedder()

    # Connexion à Qdrant
     # Connexion à Qdrant
    client = QdrantClient(url="http://localhost:6333")  # Utiliser le bon endpoint
    collection_name = "BFT_chatbot"


    # Vérifier si la collection existe déjà
    if not client.get_collections().collections or collection_name not in [c.name for c in client.get_collections().collections]:
        # Créer la collection si elle n'existe pas
        client.recreate_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=768, distance=Distance.COSINE)  # 768 car c'est la dimension de ton modèle
        )
        # Charger et ajouter des documents initiaux si la collection n'existe pas encore
        loader = CSVLoader("C:/Users/Nflak/OneDrive/Desktop/bft_chatbot/datas/faq_clcam.csv", encoding="utf-8")
        content = loader.load()

        # Diviser les documents en chunks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=10, chunk_overlap=3)
        docs = text_splitter.split_documents(documents=content)

        # Initialiser et ajouter les documents au vectorstore
        vectorstore = Qdrant(
            client=client, 
            collection_name=collection_name, 
            embeddings=embeddings
        )
        vectorstore.add_documents(docs)
    else:
        # Charger l'existant si la collection est déjà là
        vectorstore = Qdrant(
            client=client, 
            collection_name=collection_name, 
            embeddings=embeddings
        )

    # Modèle de prompt pour l'assistant
    template = """
    Tu es un assistant virtuel de BFT formé pour répondre aux questions fréquemment posées par les agents de terrain, 
    les caissiers, et les gestionnaires d'agences des institutions de microfinance. Ton objectif est de fournir des réponses 
    précises et claires pour résoudre leurs problèmes rapidement. Les questions peuvent concerner des procédures internes, 
    des problèmes techniques ou des informations sur les services de BFT. Utilise aussi les informations de ce site "https://bftgroup.co/"
    qui est le site officiel de BFT. 
    Si une question a plusieurs réponses ou options, donne toutes les réponses sans exception. 
    Si la question posée se trouve dans la FAQ, copie et renvoie exactement la réponse de la FAQ sans reformulation ni modification. 
    En plus des informations fournies dans le document, utilise tes capacités en tant que Gemini Pro pour enrichir la réponse, 
    mais assure-toi de toujours fournir les informations les plus cohérentes et pertinentes. Si tu ne trouves pas suffisamment 
    d'informations, réponds par : "Je n'ai pas assez d'informations pour répondre à cette question." Cependant, pour toutes les 
    questions basiques ou liées à toi (comme ta fonction, ton utilité, ou des questions de politesse), réponds de manière adaptée.
    context: {context}
    Question : {question}
    output
    """



    # Mémoire de conversation pour l'assistant (persistance)
    message_history = ChatMessageHistory()
    chat_memory = ConversationBufferWindowMemory(
        memory_key="chat_history",
        output_key="answer",
        chat_memory=message_history,
        k=5,
        return_messages=True
    )
    
    prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    chain_type_kwargs = {"prompt": prompt}

    # Définir le retriever pour le vectorstore
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 20})
    
    # Chaîne de conversation pour la récupération d'informations et LLM
    conversation = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=chat_memory,
        combine_docs_chain_kwargs=chain_type_kwargs,
    )
    
    return conversation

conversation = init_chat()

# Fonction pour interagir avec le chatbot
def Talker(query: str, conversation):
    output = conversation.invoke(input=query)
    response = output.get('answer')
    return response

# Exemple d'appel de la fonction Talker
print(Talker(" Que faire lorsque après enregistrement, le fichier d'enregistrement du client ne vient pas sur la plateforme Smart Saving?", conversation))


Je n'ai pas assez d'informations pour répondre à cette question.
